<a href="https://colab.research.google.com/github/lotfibenaouda/colab_monthly-caisse/blob/main/Python_colab_monthly_caisse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## First block of code
!pip install gspread-formatting  # Installs the necessary package

import gspread
from google.oauth2.service_account import Credentials
from gspread_formatting import CellFormat, NumberFormat, format_cell_range # This line should now work
import pandas as pd
from gspread_dataframe import set_with_dataframe

# Define the scope for accessing Google Sheets and Google Drive
scope = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

# Authenticate using the credentials.json file (make sure this file is uploaded to Colab)
creds = Credentials.from_service_account_file('my-project-11107-colab-1cd1b53d9983.json', scopes=scope)

# Authorize the client to interact with the Google Sheets API
client = gspread.authorize(creds)

# Correctly access the Google Sheet by name (replace with your actual sheet name)
sheet = client.open("payment_by_type_2024-09-01_2024-09-30").sheet1

# Example: Get all data from the sheet and print it
data = sheet.get_all_records()
print(data)

#cleane the file
# Delete columns in reverse order to avoid shifting
for col_index in sorted([1, 3, 4, 5, 6, 7, 8, 9, 10, 11], reverse=True):
    sheet.delete_columns(col_index)  # Delete each column

# Optional: Get the updated data after deletion to verify the result
updated_data = sheet.get_all_records()
print(updated_data)

# Define the range for column B (you can adjust this based on your data size)
range_to_format = 'B2:B'  # Assuming B1 is a header, so starting from B2

# Set the currency format for Belgian Euro (EUR)
fmt = CellFormat(
    numberFormat=NumberFormat(type='CURRENCY', pattern='€#.##0,0')
)

# Apply the formatting to the range
format_cell_range(sheet, range_to_format, fmt)

print("Column B has been formatted as Belgian Euro.")



# Second block of code

import gspread
from google.oauth2.service_account import Credentials
import pandas as pd
from gspread_dataframe import set_with_dataframe

# Authenticate and connect to the Google Sheets API
scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
creds = Credentials.from_service_account_file('my-project-11107-colab-1cd1b53d9983.json', scopes=scope)
client = gspread.authorize(creds)

# Open the Google Sheet by its title
sheet = client.open("payment_by_type_2024-09-01_2024-09-30").sheet1

# Fetch all the data from the sheet
data = sheet.get_all_records()

# Convert the data to a Pandas DataFrame for easier manipulation
df = pd.DataFrame(data)

# Ensure the 'Date' column is in datetime format and strip the time part (keeping only the date)
df['Date'] = pd.to_datetime(df['Date']).dt.date

# Group payments by 'Date', pivot them so each payment is in a new column horizontally
grouped = df.groupby('Date')['Net payments'].apply(lambda x: pd.Series(x.values)).unstack()

# Calculate the sum of net payments for each day and add as a new column
grouped['Total Net Payments'] = df.groupby('Date')['Net payments'].sum()

# Create a complete list of all days in July 2024
all_days = pd.date_range(start="2024-09-01", end="2024-09-30").date

# Merge grouped data with all days, filling missing days with 0 for net payments
grouped = pd.DataFrame(all_days, columns=['Date']).merge(grouped, on='Date', how='left').fillna(0)

# Reset the index so that 'Date' becomes a column again
grouped = grouped.reset_index(drop=True)

# Create a new sheet for the horizontal data with the sum column and missing days filled in
new_sheet = client.open("payment_by_type_2024-09-01_2024-09-30").add_worksheet(title="Horizontal_Payments_with_Sum", rows=100, cols=20)

# Write the transformed data back into the new sheet
set_with_dataframe(new_sheet, grouped)

print("Data has been transformed, missing days added, and written back to the new sheet horizontally.")



[{'Transaction ID': 7664551100761, 'Date': '2024-09-01T15:03:11+02:00', 'Order ID': 6231228514649, 'Order': '#42724', 'Payment type': 'Cash', 'Credit card': '', 'Credit card type': '', 'Billing country': '', 'Gift card ID': 0, 'Gross payments': 15.0, 'Refunds': 0.0, 'Net payments': 15.0}, {'Transaction ID': 7664588128601, 'Date': '2024-09-01T15:25:52+02:00', 'Order ID': 6231258857817, 'Order': '#42725', 'Payment type': 'Cash', 'Credit card': '', 'Credit card type': '', 'Billing country': '', 'Gift card ID': 0, 'Gross payments': 55.0, 'Refunds': 0.0, 'Net payments': 55.0}, {'Transaction ID': 7664634036569, 'Date': '2024-09-01T15:54:26+02:00', 'Order ID': 6231295983961, 'Order': '#42726', 'Payment type': 'Cash', 'Credit card': '', 'Credit card type': '', 'Billing country': '', 'Gift card ID': 0, 'Gross payments': 10.0, 'Refunds': 0.0, 'Net payments': 10.0}, {'Transaction ID': 7671841685849, 'Date': '2024-09-03T18:06:53+02:00', 'Order ID': 6237406003545, 'Order': '#42740', 'Payment type':